In [1]:
from numpy import array
from numpy import asarray
from numpy import zeros
import pandas as pd
import numpy as np

import re
import pickle 
#import mglearn
import time
# import gensim
import os
import collections
# import smart_open
import random
import string
import nltk

from nltk import Text
from nltk.tokenize import TweetTokenizer # doesn't split at apostrophes
from nltk.tokenize import regexp_tokenize
from nltk.tokenize import word_tokenize  
from nltk.tokenize import sent_tokenize 
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

from sklearn import decomposition
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score

In [2]:
from numpy import array
from numpy import asarray
from numpy import zeros
import pandas as pd
import numpy as np

import re
import pickle 
#import mglearn
import time
# import gensim
import os
import collections
# import smart_open
import random
import string
import nltk

from nltk import Text
from nltk.tokenize import TweetTokenizer # doesn't split at apostrophes
from nltk.tokenize import regexp_tokenize
from nltk.tokenize import word_tokenize  
from nltk.tokenize import sent_tokenize 
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

from sklearn import decomposition
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
from sklearn.model_selection import train_test_split

In [3]:
import keras.backend as K
import keras.layers as layers
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from keras.callbacks import ModelCheckpoint
from keras.engine import Layer
from keras.layers import CuDNNLSTM, Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Embedding, Dense, Flatten, Input
from keras.layers import SpatialDropout1D, add, concatenate
from keras.layers.convolutional import Conv1D
from keras.layers.pooling import MaxPool1D
from keras.models import Model
from keras.preprocessing import text, sequence


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split


Using TensorFlow backend.


ImportError: Traceback (most recent call last):
  File "C:\Users\Administrator\Anaconda3\envs\temp\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "C:\Users\Administrator\Anaconda3\envs\temp\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "C:\Users\Administrator\Anaconda3\envs\temp\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "C:\Users\Administrator\Anaconda3\envs\temp\lib\imp.py", line 243, in load_module
    return load_dynamic(name, filename, file)
  File "C:\Users\Administrator\Anaconda3\envs\temp\lib\imp.py", line 343, in load_dynamic
    return _load(spec)
ImportError: DLL load failed: 이 작업을 완료하기 위한 페이징 파일이 너무 작습니다.


Failed to load th

In [4]:
# def data_check():
#     data = pd.read_csv('all_emo_clean_data/all_emo_clean_data.csv')
#     data = data[['clean_text','label']]
#     print(pd.value_counts(data['label'].values, sort = False))
#     return data

In [5]:
def data_load(all_file_path):
    data = pd.read_csv(all_file_path) 
    data = data[['clean_text','label']]
    print("Data volume", data.shape)
    print()
    
    # y text label to num label 
    emo_label = list(set(data['label'].values))
    print(emo_label)

    # split train, text set 
    train, val_test = train_test_split(data, test_size=0.3, random_state=42)
    val, test = train_test_split(val_test, test_size=0.5, random_state=42)
    
    # spilit x and y 
    train_x, train_y = train["clean_text"], train["label"]
    val_x, val_y = val["clean_text"], val["label"]
    test_x, test_y = test["clean_text"], test["label"]
        
    ## y label check 
    print("Label Check")
    label_check_df = pd.DataFrame()
    label_check_df['totaldata'] = pd.value_counts(data['label'].values, sort = False)
    label_check_df['train'] = pd.value_counts(train_y, sort = False)
    label_check_df['val'] = pd.value_counts(val_y, sort = False)
    label_check_df['test'] = pd.value_counts(test_y, sort = False)
    display(label_check_df)
    
    
    print('1. load_data \n train_x.shape', train_x.shape,"\n====================")
    return train_x, train_y,val_x, val_y, test_x, test_y

In [6]:
def data_preprocissing(train_x, test_x,val_x):

    tokenizer = text.Tokenizer(split=' ')
    tokenizer.fit_on_texts(list(train_x) + list(test_x) + list(val_x))  # Make dictionary

    # Text match to dictionary.
    train_x = tokenizer.texts_to_sequences(train_x)
    test_x = tokenizer.texts_to_sequences(test_x)
    val_x = tokenizer.texts_to_sequences(val_x)

    temp_list = []
    total_list = list(train_x) + list(test_x) + list(val_x)

    for i in range(0, len(total_list)):
        temp_list.append(len(total_list[i]))

    max_len = max(temp_list)

    train_x = sequence.pad_sequences(train_x, maxlen=max_len)
    test_x = sequence.pad_sequences(test_x, maxlen=max_len)
    val_x = sequence.pad_sequences(val_x, maxlen=max_len)

    print('2. data_preprocissing \n train_x.shape', train_x.shape)
    print('max_len', max_len, "\n====================")
    return train_x, test_x, val_x, tokenizer


In [7]:
def get_coefs(word, *arr):
    return word, np.asarray(arr, dtype='float32')

In [8]:
def load_embeddings(path):
    with open(path, encoding="utf-8") as f:
        return dict(get_coefs(*line.strip().split(' ')) for line in f)

In [9]:
# Pre-trained embedding match to my dataset.
def text_to_vector(em_filename, word_index, path):
    if os.path.isfile(em_filename):
        print("Loading saved embedding matrix")
        with open(em_filename, 'rb') as rotten_file:
            embedding_matrix = pickle.load(rotten_file)
    
    else:
        # If you change your embedding.pickle file, you must make new embedding.pickle file.
        print("Making new embedding matrix")
        embedding_index = load_embeddings(path)
        embedding_matrix = np.zeros((len(word_index) + 1, 300))
        for word, i in word_index.items():
            try:
                embedding_matrix[i] = embedding_index[word]
            except KeyError:
                pass

        with open(em_filename, 'wb') as handle:
             pickle.dump(embedding_matrix, handle, protocol=pickle.HIGHEST_PROTOCOL)

    return embedding_matrix

In [10]:
# # TextCNN

#     batch_size = 32
#     num_classes = 8 
#     epochs = 8
#     num_filters = 1024
#     filter_sizes = [1, 3, 5, 7]
#     spatdrop = 0.2
#     poolsize = 2
#     train = True
    


def build_model(size, num_filters, filter_sizes, spatdrop, poolsize, train, embedding_matrix): # size 는 224
    print("4.build_model")
    print('size', size)
    print('embedding_matrix.shape', embedding_matrix.shape)
    
    print("hyper parameter setting")
    print("Num of filters",num_filters)
    print("Filter sizes",filter_sizes)
    print("SpatialDropout1D",spatdrop)
    print("Pool size",poolsize)
    print("Trainable",train)
    print("Embedding matrix size",embedding_matrix.shape)
    
    input_layer = Input(shape=(size,))
    embedding_layer = Embedding(*embedding_matrix.shape, weights=[embedding_matrix], trainable=train)(input_layer)
    embedding_layer = SpatialDropout1D(spatdrop)(embedding_layer)

    pooled_outputs = []

    for filter_size in filter_sizes:
        x = Conv1D(num_filters, filter_size, activation='relu')(embedding_layer)
        x = MaxPool1D(pool_size=poolsize)(x)
        pooled_outputs.append(x)

    merged = concatenate(pooled_outputs, axis=1)
    dense_layer = Flatten()(merged)

    result = Dense(8, activation='softmax')(dense_layer)

    model = Model(inputs=input_layer, outputs=result)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()

    return model

In [13]:
def evaluate(model, test_x, test_y):
    prediction = model.predict(test_x)
    
    # multi class classification case 
    # 1. 예측된 각 클래스의 값 확인 (한 클래스에 값이 쏠리진 않았는지 체크 )
    unique, counts = np.unique(prediction.argmax(axis=1), return_counts=True)
    predicted_label_count = dict(zip(unique, counts))
    print("predicted label count :", predicted_label_count)
    
    print("Confusion matrix")
    matrix = confusion_matrix(test_y.argmax(axis=1), prediction.argmax(axis=1))
    print(matrix)
    
    print("Classification Report")
    target_names = ['anger','anticipation','disgust','fear','joy','trust','sadness','surprise']
    report = classification_report(test_y.argmax(axis=1), prediction.argmax(axis=1), target_names=target_names)
    print(report)
    
    print("Accuracy")
    accuracy = accuracy_score(test_y.argmax(axis=1), prediction.argmax(axis=1))
    print(accuracy)
    
    print("F1 - score")
    f1score = f1_score(test_y.argmax(axis=1), prediction.argmax(axis=1), average = 'macro')
    print(f1score)
    
    return predicted_label_count, matrix, report, accuracy, f1score
    


In [14]:
if __name__ == "__main__":

    #####  hyperparameter 
    batch_size = 32
    num_classes = 8 
    epochs = 8
    num_filters = 1024
    filter_sizes = [1, 3, 5, 7]
    spatdrop = 0.2
    poolsize = 2
    train = True
    
    ## path and file name 
    
    file_path = 'C:/Users/Administrator/Anaconda3/envs/temp/EmotionEmbedding/emotion8_finalcode/'
    embedding_dir = 'C:/Users/Administrator/Anaconda3/envs/MR/glove/glove.6B.300d.txt' # pretrained model
    
    name = 'pe8_3'
    all_file_path = file_path+'data/tweet_data/all_emo_clean_data/all_emo_clean_data3.csv'
    em_layername = file_path+"embeddinglayer/layer_%s.pickle"%name # keras embedding layer name 
    em_filename = file_path+"glovematrix/glovemat_%s.pickle"%name  # save glove embedding matrix 
    word_indexname = file_path+"wordindex/wordindex_%s.pickle"%name  # save glove embedding matrix 
    
    # 1. data load / train test splilt
    train_x, train_y, val_x, val_y, test_x, test_y = data_load(all_file_path)
    
    # num label to one hot encoding 
    train_y = to_categorical(train_y)
    test_y = to_categorical(test_y)
    val_y = to_categorical(val_y)

    # 2. data preprocessing
    train_x, test_x, val_x, tokenizer = data_preprocissing(train_x, test_x, val_x)
    
    # 3. Embedding matrix 
    embedding_matrix = text_to_vector(em_filename, tokenizer.word_index, embedding_dir)
    
    # 4. Modeling 
    model = build_model(train_x.shape[1], num_filters, filter_sizes, spatdrop, poolsize, train, embedding_matrix)
    model.fit(x=train_x, y=train_y, epochs=epochs, batch_size=batch_size, validation_data=(val_x, val_y))
    
    # 5. Evaluation 
    predicted_label_count, matrix, report, accuracy, f1score = evaluate(model, test_x, test_y)
    
    # 6.1 save word index 
    worddict = tokenizer.word_index
    with open(word_indexname, 'wb') as handle:
        pickle.dump(worddict, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    # 6.2 save embedding model
    embedding_layer_matrix = model.layers[1].get_weights()[0]
    with open(em_layername, 'wb') as handle:
        pickle.dump(embedding_layer_matrix, handle, protocol=pickle.HIGHEST_PROTOCOL)
    

    

Data volume (144701, 2)

[0, 1, 2, 3, 4, 5, 6, 7]
Label Check


,totaldata,train,val,test
0,44488,31025,6733,6730
1,8089,5700,1139,1250
2,8678,6042,1309,1327
3,20012,14071,3035,2906
4,22489,15761,3341,3387
5,10697,7508,1609,1580
6,20462,14326,3071,3065
7,9786,6857,1468,1461


1. load_data 
 train_x.shape (101290,) 
2. data_preprocissing 
 train_x.shape (101290, 34)
max_len 34 
Making new embedding matrix


W1217 15:49:20.753159 25556 deprecation_wrapper.py:119] From C:\Users\Administrator\Anaconda3\envs\temp\lib\site-packages\keras\backend\tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1217 15:49:20.771101 25556 deprecation_wrapper.py:119] From C:\Users\Administrator\Anaconda3\envs\temp\lib\site-packages\keras\backend\tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1217 15:49:20.776089 25556 deprecation_wrapper.py:119] From C:\Users\Administrator\Anaconda3\envs\temp\lib\site-packages\keras\backend\tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1217 15:49:20.785064 25556 deprecation_wrapper.py:119] From C:\Users\Administrator\Anaconda3\envs\temp\lib\site-packages\keras\backend\tensorflow_backend.py:190: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_se

4.build_model
size 34
embedding_matrix.shape (83204, 300)
hyper parameter setting
Num of filters 1024
Filter sizes [1, 3, 5, 7]
SpatialDropout1D 0.2
Pool size 2
Trainable True
Embedding matrix size (83204, 300)


W1217 15:49:21.079276 25556 deprecation.py:506] From C:\Users\Administrator\Anaconda3\envs\temp\lib\site-packages\keras\backend\tensorflow_backend.py:3733: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W1217 15:49:21.108201 25556 deprecation_wrapper.py:119] From C:\Users\Administrator\Anaconda3\envs\temp\lib\site-packages\keras\backend\tensorflow_backend.py:4267: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W1217 15:49:21.162057 25556 deprecation_wrapper.py:119] From C:\Users\Administrator\Anaconda3\envs\temp\lib\site-packages\keras\optimizers.py:793: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W1217 15:49:21.245833 25556 deprecation.py:323] From C:\Users\Administrator\Anaconda3\envs\temp\lib\site-packages\tensorflow\p

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 34)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 34, 300)      24961200    input_1[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_1 (SpatialDro (None, 34, 300)      0           embedding_1[0][0]                
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 34, 1024)     308224      spatial_dropout1d_1[0][0]        
____________________________________________________________________________________________

#  finish modeling